# 👜 Hermès Handbag Price Analysis

This project analyzes the global pricing of Hermès’ most iconic handbags using user-submitted data from the **PurseForum**. The data consists of handbag prices collected at the end of each year, shared by users across various countries.

#### Check README.md for complete info.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from pyspark.sql.functions import col, split, explode
from pyspark.sql import SparkSession

# Initialize Spark session
spark = SparkSession.builder.appName("CSVExample").getOrCreate()

### Data Ingestion & Cleaning

In [19]:
source = pd.read_csv('2024-2019_raw.csv')

In [20]:
# Cleaned Dataset for Experiment and Querying
from transformation import split_and_clean
cleaned = split_and_clean(source)         #Script Developed and now collapsed into transformation.py

In [21]:
#Checking data type for database creation
cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1559 entries, 0 to 1558
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Bag_Type         1559 non-null   object 
 1   Date_Purchased   1559 non-null   object 
 2   Material_Style   1559 non-null   object 
 3   Price            1559 non-null   float64
 4   Currency         1559 non-null   object 
 5   Bag_Size         1559 non-null   object 
 6   Leather          1559 non-null   object 
 7   Exotic_Skin      1559 non-null   object 
 8   Stitching        1559 non-null   object 
 9   Limited_Edition  1559 non-null   object 
dtypes: float64(1), object(9)
memory usage: 121.9+ KB


#### Using Apache Spark to query and to check for N/A for scripts optimization
- This was used to check for any N/A Values for scripts improvement for pipeline since there are multiple ways that user spell or list the specification of their bag

In [22]:
# For querying and look at leftover, this was used to check for N/A for optimization
spark_df = spark.createDataFrame(cleaned)
spark_df.where(col("Leather") == "NaN") \
    .show(10,truncate=False)

+--------+--------------+--------------+-----+--------+--------+-------+-----------+---------+---------------+
|Bag_Type|Date_Purchased|Material_Style|Price|Currency|Bag_Size|Leather|Exotic_Skin|Stitching|Limited_Edition|
+--------+--------------+--------------+-----+--------+--------+-------+-----------+---------+---------------+
+--------+--------------+--------------+-----+--------+--------+-------+-----------+---------+---------------+



#### Final Cleaned Look 

In [23]:
#Final Cleaned Look
cleaned.head(8)

,Bag_Type,Date_Purchased,Material_Style,Price,Currency,Bag_Size,Leather,Exotic_Skin,Stitching,Limited_Edition
0,Kelly,1/2/24,Togo Retourne,8600.0,EUR,25,Togo,Regular Leather,Retourne,Regular
1,Kelly,1/2/24,Epsom Sellier,9500.0,EUR,25,Epsom,Regular Leather,Sellier,Regular
2,Kelly,1/2/24,Swift,9000.0,EUR,25,Swift,Regular Leather,Sellier,Regular
3,Kelly,1/2/24,Mysore Chèvre Sellier,9600.0,EUR,25,Mysore Chèvre,Regular Leather,Sellier,Regular
4,Kelly,1/2/24,Shiny Crocodile Sellier,42400.0,EUR,25,Exotic,Croc,Sellier,Regular
5,Kelly,3/20/24,Togo Retourne,8600.0,EUR,25,Togo,Regular Leather,Retourne,Regular
6,Kelly,9/30/24,Chevre/Epsom/Tadelakt Casaque Étoilée,11500.0,EUR,25,Chevre,Regular Leather,Sellier,Casaque
7,Kelly,10/21/24,Togo Touch Lizard,120000.0,CNY,25,Togo,Lizard,Sellier,Touch


## Currency Conversion
- Converting to USD from origin Currency using the dates of purchased.
- Used Basic Tier for 5000 Requests to Pull Data

In [24]:
#Checking for Valid ISO type for Currency for Valid Conversion
cleaned['Currency'].unique()

array(['EUR', 'CNY', 'USD', 'GBP', 'SGD', 'CAD', 'THB', 'CHF', 'AUD',
       'HKD', 'CZK', 'SEK', 'AED', 'KRW', 'JPY', 'MOP', 'SAR', 'MYR',
       'PHP', 'QAR', 'MXN'], dtype=object)

In [ ]:
from currency_converter import convert_historical_prices
ready = convert_historical_prices(cleaned)
ready.head()

### Ran out of API Requests Call. But this is the output after Converting 1600 Rows. 
<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Bag_Type</th>
      <th>Date_Purchased</th>
      <th>Material_Style</th>
      <th>Price</th>
      <th>Currency</th>
      <th>Bag_Size</th>
      <th>Leather</th>
      <th>Exotic_Skin</th>
      <th>Stitching</th>
      <th>Limited_Edition</th>
      <th>Price_USD</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>Kelly</td>
      <td>1/2/24</td>
      <td>Togo Retourne</td>
      <td>8600.0</td>
      <td>EUR</td>
      <td>25</td>
      <td>Togo</td>
      <td>Regular Leather</td>
      <td>Retourne</td>
      <td>Regular</td>
      <td>9409.60</td>
    </tr>
    <tr>
      <th>1</th>
      <td>Kelly</td>
      <td>1/2/24</td>
      <td>Epsom Sellier</td>
      <td>9500.0</td>
      <td>EUR</td>
      <td>25</td>
      <td>Epsom</td>
      <td>Regular Leather</td>
      <td>Sellier</td>
      <td>Regular</td>
      <td>10394.33</td>
    </tr>
    <tr>
      <th>2</th>
      <td>Kelly</td>
      <td>1/2/24</td>
      <td>Swift</td>
      <td>9000.0</td>
      <td>EUR</td>
      <td>25</td>
      <td>Swift</td>
      <td>Regular Leather</td>
      <td>Sellier</td>
      <td>Regular</td>
      <td>9847.26</td>
    </tr>
    <tr>
      <th>3</th>
      <td>Kelly</td>
      <td>1/2/24</td>
      <td>Mysore Chèvre Sellier</td>
      <td>9600.0</td>
      <td>EUR</td>
      <td>25</td>
      <td>Mysore Chèvre</td>
      <td>Regular Leather</td>
      <td>Sellier</td>
      <td>Regular</td>
      <td>10503.74</td>
    </tr>
    <tr>
      <th>4</th>
      <td>Kelly</td>
      <td>1/2/24</td>
      <td>Shiny Crocodile Sellier</td>
      <td>42400.0</td>
      <td>EUR</td>
      <td>25</td>
      <td>Exotic</td>
      <td>Croc</td>
      <td>Sellier</td>
      <td>Regular</td>
      <td>46391.54</td>
    </tr>
    <tr>
      <th>...</th>
      <td>...</td>
      <td>...</td>
      <td>...</td>
      <td>...</td>
      <td>...</td>
      <td>...</td>
      <td>...</td>
      <td>...</td>
      <td>...</td>
      <td>...</td>
      <td>...</td>
    </tr>
    <tr>
      <th>1554</th>
      <td>Evelyne</td>
      <td>1/1/19</td>
      <td>Sellier Epsom</td>
      <td>3330.0</td>
      <td>GBP</td>
      <td>33</td>
      <td>Epsom</td>
      <td>Regular Leather</td>
      <td>Sellier</td>
      <td>Regular</td>
      <td>4246.91</td>
    </tr>
    <tr>
      <th>1555</th>
      <td>Evelyne</td>
      <td>1/22/19</td>
      <td>Sellier Epsom</td>
      <td>5450.0</td>
      <td>CAD</td>
      <td>33</td>
      <td>Epsom</td>
      <td>Regular Leather</td>
      <td>Sellier</td>
      <td>Regular</td>
      <td>4079.80</td>
    </tr>
    <tr>
      <th>1556</th>
      <td>Evelyne</td>
      <td>1/22/19</td>
      <td>Sellier Hunter Cowhide</td>
      <td>5350.0</td>
      <td>USD</td>
      <td>33</td>
      <td>Hunter</td>
      <td>Regular Leather</td>
      <td>Sellier</td>
      <td>Regular</td>
      <td>5350.00</td>
    </tr>
    <tr>
      <th>1557</th>
      <td>Evelyne</td>
      <td>2/4/19</td>
      <td>Sellier Hunter</td>
      <td>6000.0</td>
      <td>USD</td>
      <td>33</td>
      <td>Hunter</td>
      <td>Regular Leather</td>
      <td>Sellier</td>
      <td>Regular</td>
      <td>6000.00</td>
    </tr>
    <tr>
      <th>1558</th>
      <td>Evelyne</td>
      <td>6/12/19</td>
      <td>Sellier Hunter</td>
      <td>6000.0</td>
      <td>USD</td>
      <td>33</td>
      <td>Hunter</td>
      <td>Regular Leather</td>
      <td>Sellier</td>
      <td>Regular</td>
      <td>6000.00</td>
    </tr>
  </tbody>
</table>
<p>1559 rows × 11 columns</p>
</div>

## Collapsed into a single script to run all of this

In [ ]:
from pipeline import main
main()

### Due to no more API Call but script ran successfully and data has been inserted into PostgreSQL
2025-04-09 11:48:43,892 [INFO] Starting Hermes data pipeline...
2025-04-09 11:48:48,689 [INFO] SUCCESS: Loaded data from '2024-2019_raw.csv' — 1559 rows.
2025-04-09 11:48:48,712 [INFO] SUCCESS: Data Cleaning and Transformation Completed
✅ PostgreSQL connection successful!
2025-04-09 11:48:53,045 [ERROR] FAILED: Currency Conversion not Complete. Please Check API Settings/Subscriptions


## PostgreSQL Server Connection & Querying
- I want to store the conversion data and the rest of the data for future reference that I can easily pull for any purpose

### Testing Connection to Postgres

In [25]:
# Testing Connection to Server 
from con_checker import test_postgres_connection
test_postgres_connection()

✅ PostgreSQL connection successful!


True

### Creating Engine & Querying for Test

In [40]:
# Creating Engine from PostgreSQL Server
from to_postgre import connect_postgres

# Engine Connection
engine = connect_postgres(user, password, host, port, db)

#Reading from PostgreSQL Server to Dataframe for EDA
df_read = pd.read_sql("SELECT * FROM hermes.handbag",con=engine, index_col='id')
df_read.head(5)

,Date_Purchased,Bag_Type,Bag_Size,Material_Style,Price,Currency,Price_USD,Stitching,Leather,Exotic_Skin,Limited_Edition
id,,,,,,,,,,,
10,2024-01-02,Kelly,25,Togo Retourne,8600.0,EUR,9409.60,Retourne,Togo,Regular Leather,Regular
11,2024-01-02,Kelly,25,Epsom Sellier,9500.0,EUR,10394.33,Sellier,Epsom,Regular Leather,Regular
12,2024-01-02,Kelly,25,Swift,9000.0,EUR,9847.26,Sellier,Swift,Regular Leather,Regular
13,2024-01-02,Kelly,25,Mysore Chèvre Sellier,9600.0,EUR,10503.74,Sellier,Mysore Chèvre,Regular Leather,Regular
14,2024-01-02,Kelly,25,Shiny Crocodile Sellier,42400.0,EUR,46391.54,Sellier,Exotic,Croc,Regular


##